In [ ]:
#covert data from parquet
from fastparquet import ParquetFile
filename = "dbpedia_09_2022.parquet"
pf = ParquetFile(filename)
df = pf.to_pandas()
df.to_csv("dbpedia_09_2022.txt",sep='\t',index=False)
print("OK")

In [ ]:
#write DB file with correctly format

file_rel_DB ='dbpedia_09_2022.txt'
file_rel_write_sameas='sameas.txt'
file_rel_DB_without_sameas_error ='dbpedia_09_2022_version_merge.txt'
file_rel_DB_without_error ='dbpedia_09_2022_version.txt'

filerel_write_sameas = open(file_rel_write_sameas, 'w', encoding='utf-8')
filerel_DB_without_error = open(file_rel_DB_without_error ,'w', encoding='utf-8')
filerel_DB_without_sameas_error = open(file_rel_DB_without_sameas_error, 'w', encoding='utf-8')

def read_triple_dbp_raw(file_path):
    num=0
    num1=0
    num2=0
    outputsame=''
    outputwithouterror=''
    outputwithouterrorsameas=''
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            
            line = line.strip('\n').split()
            if len(line) != 3:
                continue
            s = line[0].lstrip('<').rstrip('>')
            p = line[1].lstrip('<').rstrip('>')
            o = line[2].lstrip('<').rstrip('>')
            #write in to version1
            num=num+1
            outputwithouterror ='<'+ s +'>'+'\t'+'<'+ p +'>'+'\t'+'<'+ o +'> .''\n'
            filerel_DB_without_error.write(outputwithouterror)
            if 'http://www.w3.org/2002/07/owl#sameAs' in p  and 'http://www.wikidata.org/entity'  in o:
                num1=num1+1
                outputsame='<'+ s +'>'+'\t'+'<'+ p +'>'+'\t'+'<'+ o +'> .''\n'
                filerel_write_sameas.write(outputsame)
            if 'http://www.w3.org/2002/07/owl#sameAs' not in p:
                num2=num2+1
                outputwithouterrorsameas='<'+ s +'>'+'\t'+'<'+ p +'>'+'\t'+'<'+ o +'> .''\n'
                filerel_DB_without_sameas_error.write(outputwithouterrorsameas)
                
        print(num)
        print(num1)
        print(num2)
                

            
read_triple_dbp_raw(file_rel_DB)


#758227396 without error
#33860047 same 
#724367349
#724367311
#616564612 witout same and error
#616564604


In [ ]:
#delete 1st line from dbpedia_09_2022_version.txt and dbpedia_09_2022_version_merge.txt
#because the 1st was from dbpedia_09_2022.txt，were the column names of the table <subjetc> <relation> <object>
sed -i '1d' dbpedia_09_2022_version_merge.txt

sed -i '1d' dbpedia_09_2022_version.txt

In [ ]:
#mergeentities
file_rel_in_WIKI= 'data.txt'
file_rel_sameAS= 'sameas.txt'
file_rel_in_DB= 'dbpedia_09_2022_version_merge.txt'
filewrite = open('mergeentities.txt', 'a', encoding='utf-8')


def read_DBandWIKI(file_path):
    dictwiki=dict()
  #(DB,wiki)
    num=0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip(' .\n').split()
            if len(line) != 3:
                
                continue
            num=num+1
            s = line[0].lstrip('<').rstrip('>') #db
            p = line[1].lstrip('<').rstrip('>')
            o = line[2].lstrip('<').rstrip('>')#wiki
            dictwiki[s]=o
            #dictwikibackforward[o]=s    
            
            """
            keys = dictwiki.keys()
            if s not in keys: #(DB,wiki)
            """
                #listtriplebackforward=set()
               
                #dictwiki[s]=o
        return dictwiki

def read_WIKIandDB(file_path):
    #(Wiki,db)
    dictwikibackforward=dict()
    num=0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip(' .\n').split()
            if len(line) != 3:
                continue
            num=num+1
            s = line[0].lstrip('<').rstrip('>') #DB
            p = line[1].lstrip('<').rstrip('>')
            o = line[2].lstrip('<').rstrip('>') #wiki
            """
            keysbackforward = dictwikibackforward.keys()
            if o not in keysbackforward: #(wiki,db)
                #listtriplebackforward=set()
                """
            dictwikibackforward[o]=s    
            
        #(wiki,db)
    return dictwikibackforward    

#replace
def replace_entity_in_DB(file_path,dictbackforward):
#(wiki,db)
    numoindata=0
    numberout=0
    output=''
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            numoindata=numoindata+1
            line = line.strip(' .\n').split()
            if len(line) != 3:
                continue
            s = line[0].lstrip('<').rstrip('>')# wiki
            p = line[1].lstrip('<').rstrip('>')
            o = line[2].lstrip('<').rstrip('>') #wiki
            
                #(wiki,db)

            rl=str(p).replace("http://www.wikidata.org/","http://embedding.cc/")
                
                #replacevaluehead=set()
               
            head=str(s).replace("http://www.wikidata.org/","http://embedding.cc/")
               
               
                #double check if replacevalue in dicforward keys
                
               # print("the tail entity should be replaced is "+o)
                
                
            tail=str(o).replace("http://www.wikidata.org/","http://embedding.cc/")
               
            output ='<'+ head +'>'+'\t'+'<'+ rl +'>'+'\t'+'<'+ tail +'> .''\n'
            numberout=numberout+1
               
            filewrite.write(output)
        print("the match number is" + str(numberout))

def replace_entity_into_WIKI(file_path,dictforward):
#(DB,wiki)
#
    numoindata=0 
    numberout=0
    output=''
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip(' .\n').split()
            if len(line) != 3:
                continue
            s = line[0].lstrip('<').rstrip('>')# db
            p = line[1].lstrip('<').rstrip('>')
            o = line[2].lstrip('<').rstrip('>') # db
            numoindata=numoindata+1
            uri=p.split('/')
            newuri=p.replace(uri[2],"embedding.cc")
           
      
            
            if s in dictforward.keys() or o in dictforward.keys():
                
                replacevaluehead=dictforward.get(s) #wik
                if replacevaluehead is None:
                    head=str(s).replace("http://dbpedia.org/","http://embedding.cc/")
                else:
                    head=str(replacevaluehead).replace("http://www.wikidata.org/","http://embedding.cc/")
              
                replacevaluetail=dictforward.get(o)
                if replacevaluetail is None:
                    tail=str(o).replace("http://dbpedia.org/","http://embedding.cc/")
                else:

                    tail=str(replacevaluetail).replace("http://www.wikidata.org/","http://embedding.cc/")
            #if s in dictforward.keys() or o in dictforward.keys():
            else:
                head=str(s).replace("http://dbpedia.org/","http://embedding.cc/")
                tail=str(o).replace("http://dbpedia.org/","http://embedding.cc/")
            output ='<'+ head +'>'+'\t'+'<'+ newuri +'>'+'\t'+'<'+ tail +'> .''\n'
            numberout=numberout+1
            filewrite.write(output)
        print("the raw number in  DB is" + str(numoindata))
        print("the new number in  DB is" + str(numberout))
        
    
                #str(output).replace('http://dbpedia.org/','http://embedding.cc/')
                #double check if replacevalue in dicforward key

#DBreaderbackforward=read_DB_in_map2(file_rel_DB)
DBreaderforward=read_DBandWIKI(file_rel_sameAS)
DBreaderbackforward=read_WIKIandDB(file_rel_sameAS)

#replace_entity_in_DB(file_rel_wiki,DBreaderbackforward)
replace_entity_in_DB(file_rel_in_WIKI,DBreaderbackforward) #(wiki,db)
replace_entity_into_WIKI(file_rel_in_DB,DBreaderforward)
filewrite.close()

#read match from theb DB sameaas

In [ ]:
#caculate the triples
def readline_count(file_name):
      with open(file_name, 'r', encoding='utf-8') as file:
        num=0
        for line in file:
           
            line = line.strip(' .\n').split()
            if len(line) != 3:
                continue
            num=num+1
        print(num)
readline_count('mergeentities.txt')

In [ ]:
#degree
from collections import defaultdict
def degree(file_name):
    kg_degree = defaultdict(lambda: 0)
    with open(file_name) as file:
        #data = file.readlines()
       # print("***Train*** Number of triples: ", len(data))
     for triple in file:
            triple = triple.strip(' .\n').split()
            e1 = triple[0].lstrip('<').rstrip('>')
            r = triple[1].lstrip('<').rstrip('>')
            e2 = triple[2].lstrip('<').rstrip('>')
            kg_degree[e1] += 1
            kg_degree[e2] += 1
    return kg_degree

import numpy as np
degrees = degree("mergeentities.txt") ## Replace by KG file name here. In our case, you will do it for 3 KGs: mergeentities.txt, DBpedia.txt, Wikidata.txt
print("Avg. degree:", np.array(list(degrees.values())).mean())

#merge Avg. degree: 14.288348430923302

#db Avg. degree: 10.403151794296798


In [ ]:
def kg_size(file_name):
    E = set() # entities
    R = set() # relations
    with open(file_name) as file:
        #data = file.readlines()
        for triple in file:
            triple = triple.strip(' .\n').split()
           
            e1 = triple[0].lstrip('<').rstrip('>')# db
            r = triple[1].lstrip('<').rstrip('>')# db
            e2 = triple[2].lstrip('<').rstrip('>')# db
            
            E.update({e1, e2})
            R.add(r)
    print(f"#Entities: {len(E)}, #Relations: {len(R)}")
#kg_size('data.txt')
kg_size('mergeentities.txt')
##Entities: 145768777, #Relations: 13784
#Entities: 179759154, #Relations: 15219